<a href="https://colab.research.google.com/github/fabricecordelieres/Colab-FRAP_Script/blob/main/FRAP_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FRAP Analysis**

---
## **Data organization**
This notebook takes as an input a single zip file containing several excel files generating using an ImageJ data extraction toolset (one file per dataset). Each Xls file is expected to contain one columns per analyzed ROI where values corresponds to the following normalization of fluorescence:
* "MM_Roi_XX": Double normalized fluorescence for ROI_XX.
* "Full_Scale_MM_Roi_XX": Full scale, double normalized fluorescence for ROI_XX.
* "Time_sec": Extracted timestamp.

Additionnal columns may appear but won't be taken care of.

---
## **How to run this script**

1.   Have all you data ready: the toolset should have generated a **"_dataToUpload.zip"** file in the output folder: locate it and get ready to select it when asked.
2.   Run each step: press the play button, on the left side of the relevent cells
  1. **Step 1** is non interactive: point at the \[   \] mark. It will change to a play button. Press play and wait for a green tick mark to appear on the left.
  2. **Step 2** is interactive: if applicable, unfold the cell and point at the \[   \] mark. It will change to a play button. Press play: a **"Select file"** button should appear below the cell. Use it to navigate your drive and locate the zip file. The selected file will be uploaded to the swap space of the Google Colab environment, will be unzip.
  3. **Step 3** is non interactive: point at the \[   \] mark. It will change to a play button. Press play and wait for a green tick mark to appear on the left.  During this step, data compilation will occur. The resulting files will be zipped and directly downloaded to your computer.



#Step 1: Prepare the environment for execution
_Simply execute the two following cells either in one row or individually_

In [ ]:
#@markdown #Step 1.1: Import librairies
#@markdown _Simply execute the following cell_

from google.colab import files
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
from matplotlib.backends.backend_pdf import PdfPages
import zipfile
import shutil
from tqdm import tqdm

In [ ]:
#@markdown #Step 1.2: Define procedures on how to deal with data

#@markdown _Simply execute the following cell_

#------------------------------------------------------------------------------
def list_xls_files(directory):
  """Lists all XLS files in a given directory.

  Args:
    directory: The directory to search for XLS files.

  Returns:
    A list of strings, where each string is the full path to an XLS file.
  """
  xls_files = []
  for filename in os.listdir(directory):
    if filename.endswith(".xls") or filename.endswith(".xlsx"):
      xls_files.append(os.path.join(directory, filename))
  return xls_files

#------------------------------------------------------------------------------
# Define the single exponential function
def single_exponential_func(x, I0, a, b):
  '''Single exponential function

  Args:
    x: x value
    I0: I0 value
    a: a value
    b: b value

  Returns:
    I0-a*exp(-b*x)
  '''
  return I0 - a * np.exp(-b * x)

#------------------------------------------------------------------------------
# Define the double exponential function
def double_exponential_func(x, I0, a, b, c, d):
  '''Double exponential function

  Args:
    x: x value
    I0: I0 value
    a: a value
    b: b value
    c: c value
    d: d value

  Returns:
    I0-a*exp(-b*x)-c*exp(-d*x)
  '''
  return I0 - a * np.exp(-b * x) - c * np.exp(-d * x)

#------------------------------------------------------------------------------
# Define goodness of fitting
def goodness_of_fit(y_data, fitted_y):
  '''Computes goodness of fit using R-squared
  Args:
    y_data: original data
    fitted_y: fitted data

  Returns:
    R-squared value
  '''
  ss_res = np.sum((y_data - fitted_y) ** 2)
  ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
  r_squared = 1 - (ss_res / ss_tot)
  return r_squared

#------------------------------------------------------------------------------
def find_x_at_half_max_double_exponential(x_data, I0, a, b, c, d):
    """
    Finds the x-value where the double exponential function is at half its maximum.

    Args:
        x_data: Array of x-values.
        I0, a, b, c, d: Parameters of the double exponential function.

    Returns:
        The x-value at half maximum, or None if no solution is found.
    """

    y_max = I0
    half_max = y_max / 2

    # Define the function to solve for the root (where f(x) - half_max = 0)
    def equation_to_solve(x):
        return double_exponential_func(x, I0, a, b, c, d) - half_max

    # Initial guess for x (you might need to adjust this based on your data)
    initial_guess = x_data[np.argmin(np.abs(double_exponential_func(x_data, I0, a, b, c, d)-half_max))]

    try:
      x_half_max = fsolve(equation_to_solve, initial_guess)[0]
      return x_half_max
    except RuntimeError:
      print("Error - fsolve failed to find a solution")
      return None

#------------------------------------------------------------------------------
def fit_dataset_exponential(dataset_dir, dataset_name, is_single):
  '''

  Args:
    dataset_dir: The directory containing the dataset.
    dataset_name: The name of the dataset.
    is_single: True if the fit should be performed based on a single exponential, otherwise will be performed on a double exponential.

  Returns:
    out_path: The path to the output directory.

  '''
  # Set fitting type attribute
  fitting_type="_single-exponential" if is_single else "_double-exponential"
  fitting_type_legend="Single Exponential" if is_single else "Double Exponential"

  # Defines basename
  basename=dataset_name.replace('_Quantif_Norm.xls', '')

   # Open file
  df = pd.read_csv(os.path.join(dataset_dir, dataset_name), sep='\t')

  # Create output dir
  out_path=os.path.join(dataset_dir, basename)
  if not os.path.exists(out_path):
    os.mkdir(out_path)

  # Select columns starting with "Time_sec" or "Full_Scale"
  selected_columns = [col for col in df.columns if col.startswith('Time_sec') or col.startswith('MM_') or col.startswith('Full_Scale')]
  new_df = df[selected_columns]

  # Create lists to store fitted data and parameters
  fitted_data = []
  fitting_parameters = []
  columns_names=['Time_sec']

  with PdfPages(os.path.join(out_path, basename+fitting_type+'_plots.pdf')) as pdf:
      for col in [col for col in new_df.columns if  col.startswith('MM_') or col.startswith('Full_Scale')]:
          # Find the index of the largest drop
          data = new_df[col].values
          drop_indices = np.where(np.diff(data) < -50)[0]
          if len(drop_indices) > 0:
              max_drop_index = drop_indices[0]
              x_data = new_df['Time_sec'][max_drop_index:]
              y_data = new_df[col][max_drop_index:]

              try:

                  if(is_single):
                    params, covariance = curve_fit(single_exponential_func, x_data, y_data, p0=[100, 90, 0.015])
                    I0, a, b = params
                    fitted_y = single_exponential_func(x_data, I0, a, b)
                    #Extracted parameters
                    t_half=np.log(2)/b
                    mobile_fraction=100*a/(100-(I0-a)) # Used 100 instead of 1 cf normalisation between [0; 100] and not [0; 1]
                  else:
                    params, covariance = curve_fit(double_exponential_func, x_data, y_data, p0=[100, 60, 0.01, 30, 0.02])
                    I0, a, b, c, d = params
                    fitted_y = double_exponential_func(x_data, I0, a, b, c, d)
                    #Extracted parameters
                    t_half=find_x_at_half_max_double_exponential(x_data, I0, a, b, c, d)#find_x_at_half_max(x_data.to_numpy, fitted_y.to_numpy, I0)
                    mobile_fraction=100*(a+c)/(100-(I0-a-c)) # Used 100 instead of 1 cf normalisation between [0; 100] and not [0; 1]

                  # Get goodness of fitting
                  r_squared = goodness_of_fit(y_data, fitted_y)



                  columns_names.append(col)
                  fitted_data.append(fitted_y)

                  if(is_single):
                    fitting_parameters.append([col, I0, a, b, r_squared, t_half, mobile_fraction])
                  else:
                    fitting_parameters.append([col, I0, a, b, c, d, r_squared, t_half, mobile_fraction])

                  plt.figure(figsize=(10, 6))
                  plt.plot(x_data, y_data, label='Data')
                  plt.plot(x_data, fitted_y, label=fitting_type_legend+' Fit', color='red')
                  plt.xlabel('Time_sec')
                  plt.ylabel(col)
                  plt.title(f'{col} vs. Time_sec ('+fitting_type_legend+' Fit)')
                  plt.text(0.05, 0.95, f"R-squared: {r_squared:.4f}", transform=plt.gca().transAxes) #Display R-squared on the plot
                  plt.legend()
                  plt.grid(True)
                  pdf.savefig()
                  plt.close()

              except RuntimeError:
                  print(f"Error - curve_fit failed for {col}")
                  fitting_parameters.append([col, "Error", "Error", "Error", "Error", np.nan, np.nan])
          else:
              print(f"No significant drop found for {col}")
              fitting_parameters.append([col, "No drop", "No drop", "No drop", "No drop", np.nan, np.nan])

  # Create and save Excel files
  fitted_data_df = pd.DataFrame(fitted_data).T
  fitted_data_df.insert(0, 't', new_df['Time_sec'])
  fitted_data_df.columns = columns_names
  fitted_data_df.to_excel(os.path.join(out_path, basename+fitting_type+'_fitted_values.xlsx'), index=False)

  if(is_single):
    fitting_parameters_df = pd.DataFrame(fitting_parameters, columns=['ROI', 'I0', 'a', 'b', 'R2', 't_half', 'mobile_fraction'])
  else:
    fitting_parameters_df = pd.DataFrame(fitting_parameters, columns=['ROI', 'I0', 'a', 'b', 'c', 'd', 'R2', 't_half', 'mobile_fraction'])

  fitting_parameters_df.to_excel(os.path.join(out_path, basename+fitting_type+'_fitting_parameters.xlsx'), index=False)

  return out_path

#------------------------------------------------------------------------------
def unzip_file(zip_filepath, extract_dir):
  """Unzips a zip file to a specified directory.

  Args:
    zip_filepath: The path to the zip file.
    extract_dir: The directory to extract the contents to.
  """
  with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

#------------------------------------------------------------------------------
def create_zip_from_folders(folder_list, zip_filename, remove_folder):
  """Creates a zip archive containing all folders from a given list.

  Args:
    folder_list: A list of folder paths to include in the zip archive.
    zip_filename: The name of the output zip file.
    remove_folder: True to erase the original folders after zipping.
  """
  with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for folder in folder_list:
      for root, _, files in os.walk(folder):
        for file in files:
          zipf.write(os.path.join(root, file),
                     os.path.relpath(os.path.join(root, file),
                                     os.path.join(folder, '..')))
  if remove_folder:
    for folder in folder_list:
      shutil.rmtree(folder)



#Step 2: Upload data
_Simply execute the following cell, then select the file to upload_

**NB:** This cell should be unfolded in order to see the file selection box !

In [ ]:
# Display a dialog box to allow uploading the data
uploaded = files.upload()

#Step 3: Process data
_Simply execute the following cell: once processing has been performed, the output file should automatically be downloaded to your computer_


In [ ]:
#Get the name of the (first) uploaded file
uploaded_file=next(iter((uploaded)))

# Build the full paths for uploaded and unzipped data
uploaded_file_path=os.path.join('/content/', uploaded_file)
unzip_folder=os.path.join('/content/', uploaded_file.replace('.zip', '/'))

# Unzip the first file into the content folder
unzip_file(uploaded_file_path, unzip_folder)

# Remove uploaded file
os.remove(uploaded_file)

# Get the list of all the XLS files
list_xls=list_xls_files(unzip_folder)
created_folders=[]

# Analyze data
for xls in tqdm(list_xls, desc="Processing file"):
  tqdm.write(f"Processing file: {os.path.basename(xls)}")
  created_folders.append(fit_dataset_exponential(unzip_folder, os.path.basename(xls), True))
  fit_dataset_exponential(unzip_folder, os.path.basename(xls), False)

# Zip the data
create_zip_from_folders(created_folders, 'Fitted_data.zip', True)

# Remove the data, once zipped
shutil.rmtree(unzip_folder)

# Automatically download the file
files.download('Fitted_data.zip')

#Step 4: What's in the output ?

Once the script has finished processing the data, all outputs will be automatically zipped (Fitted_data.zip) and downloaded. It contains one folder per input dataset, named after the dataset. Each folder contains the following files:
- **DATASET-NAME_single-exponential_plots.pdf:** for data where fitting against a single exponential was possible, displays one page per ROI, overlaying as a graph the fitted data to the original data.
- **DATASET-NAME_single-exponential_fitted_values.xlsx:** for double normalized and full scale data, displays the modelized the values (when fitting a single exponential was possible).
- **DATASET-NAME_single-exponential_fitting_parameters.xlsx:** for double normalized and full scale data, displays the parameters for the single exponential model, a goodness of fitting value and estimation of both the t_half and mobile fraction (when fitting was possible).
- **DATASET-NAME_double-exponential_plots.pdf:** for data where fitting against a double exponential was possible, displays one page per ROI, overlaying as a graph the fitted data to the original data.
- **DATASET-NAME_double-exponential_fitted_values.xlsx:** for double normalized and full scale data, displays the modelized the values (when fitting a double exponential was possible).